In [1]:
# Import Required Libraries
%pip install seaborn 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, classification_report


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 3.9 MB/s eta 0:00:004.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [seaborn]━━━ 3/4 [seaborn]
Note: you may need to restart the kernel to use updated packages.


2025-07-22 00:35:32.854371: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-22 00:35:32.859516: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-22 00:35:32.878781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753137332.913557   32136 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753137332.923690   32136 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753137332.946965   32136 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

ModuleNotFoundError: No module named 'sklearn'

# Model Evaluation on PlantVillage Dataset

This notebook evaluates the trained model using the test set. It includes metrics, confusion matrix, ROC curves, and a classification report.

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 16

In [8]:
import tensorflow_datasets as tfds
from preprocessing import preprocess, preprocess_with_ohe

# Load Trained Model and Test Data
arch_choice = 1  # Choose the architecture version
if arch_choice not in [1, 2, 3]:
    raise ValueError("Invalid architecture choice")

#model = keras.models.load_model(f'best_model_v{arch_choice}.h5')
model = keras.models.load_model(f'best_model_wo_augmentation.h5')


ds_info = tfds.builder('plant_village').info
ds_test = tfds.load('plant_village', split='train[95%:]', as_supervised=True)

test_ds = ds_test.map(lambda image, label: preprocess(image, label, ds_info, IMG_SIZE)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
class_names = ds_info.features['label'].names


ImportError: cannot import name 'preprocess_with_ohe' from 'preprocessing' (d:\Git\progetto-daml\preprocessing.py)

In [9]:
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
    download=False
)

AssertionError: Dataset plant_village: could not find data in C:\Users\Utente\tensorflow_datasets. Please make sure to call dataset_builder.download_and_prepare(), or pass download=True to tfds.load() before trying to access the tf.data.Dataset object.

In [ ]:
class_names = ds_info.features['label'].names
families   = sorted({n.split('___')[0] for n in class_names})
if FAM_SPLIT:
    split_labels = families
    family_map = [
        families.index(name.split('___')[0])
        for name in class_names
    ]
    family_map = tf.constant(family_map, dtype=tf.int32)
else:
    split_labels = class_names

NUM_LAB = len(split_labels)

In [ ]:
def folder_to_fam_ohe(image, folder_label):
    fam_idx = tf.gather(family_map, folder_label)
    fam_ohe = tf.one_hot(fam_idx, NUM_LAB)
    return image, fam_ohe

In [ ]:
if FAM_SPLIT:
    ds_test_ohe = (
      ds_test
      .map(folder_to_fam_ohe)
      .map(preprocess)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
    )
else:
    ds_test_ohe = (
      ds_test
      .map(preprocess_with_ohe)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
    )

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('best_model_wo_augmentation.h5')

In [ ]:
# Generate Predictions on Test Set
y_true = []
y_pred = []
y_score = []
for images, labels in ds_test_ohe:
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_score.append(preds)
y_score = np.concatenate(y_score)


In [ ]:
# Calculate Evaluation Metrics (Accuracy, Precision, Recall, F1)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1-score: {f1:.6f}")

In [ ]:
# # Plot Confusion Matrix
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred, normalize='true')

#Plot confusion matrix
plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=False, fmt='d', cmap='viridis', xticklabels=split_labels, yticklabels=split_labels)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix (Test Set)')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
'''
import matplotlib.pyplot as plt
from plotting import plot_confusion_matrix

plt.figure()  # Set a larger figure size for better readability
plot_confusion_matrix(
    np.array(y_true),
    np.array(y_pred),
    classes=class_names,
    normalize=True,
    title="Confusion Matrix (Test Set)",
    cmap="plasma"
)
plt.tight_layout()  # Ensure labels and ticks are not cut off
'''

In [ ]:
# Plot ROC Curves for Each Class
n_classes = y_score.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(np.eye(n_classes)[y_true][:, i], y_score[:, i])
    roc_auc[i] = roc_auc_score(np.eye(n_classes)[y_true][:, i], y_score[:, i])
plt.figure(figsize=(12, 12))
auc_and_idx = sorted([(roc_auc[i], i) for i in range(n_classes)], reverse=True)
for auc, i in auc_and_idx:
    plt.plot(fpr[i], tpr[i], label=f'{split_labels[i]} (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], 'r--', lw=2, label='Random Classifier (AUC = 0.5)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - One vs Rest (Test Set)')
plt.legend(fontsize='small', bbox_to_anchor=(1.05, 1), loc='best')
plt.show()


In [ ]:
# from plotting import make_roc
# fig, ax = plt.subplots(figsize=(16, 8))
# make_roc(np.eye(len(class_names))[y_true], y_score, class_names)
# # Move legend to a dedicated column outside the plot
# plt.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), fontsize="x-small", frameon=False)
# plt.subplots_adjust(right=0.75)  # Make space for the legend
# plt.show()


In [ ]:
# Display Classification Report
report = classification_report(y_true, y_pred, target_names=split_labels)
print(report)
